<h1> <font color=steelblue>Run DBSCAN on Cluster</font></h1>

<h3><font color=darkcyan> USGS NHD 2.1 </font></h3>

<p> This work is performed on a dataset of USGS field mesurements of channel hydraulics originally collected by Barber and Gleason (2018) joined to NHD, resulting in an expanded version of the dataset used in Brinkerhoff, et al. (2019)
    <br><br>
    Stream Order <br>
    Slope <br>
    Distance Downstream <br>
    Maximum Grain Size Entrained (Henderson, 1966) <br>
    Shear Stress <br>
    Froude Number <br>
    Drainage Area <br>
    Unit Stream Power <br>
    Channel Shape <br>
    Channel Width <br>
    Channel Depth <br>
    Channel Velocity <br>
    Bankfull Shape <br>
    Bankfull Width <br>
    Bankfull Depth <br>
    Bankfull Velocity <br>
    Reach Type (i.e. perennial, intermittent, lake/wetland/reservoir, canal, or connector).<br>
    Sinuosity
    <br><br>It must be stressed that these joined variables exist at the reach-scale, while the field measurements exist at the cross-section scale. Also, these results are only reflective of the hydrology/geomorphology of river reaches measured by the USGS, generally at streamagauges.</p>

<h3><font color=darkcyan> Lakes/Reservoirs Preprocessing on NHD </font></h3>
<p> NHD for some reason has assigned many main stem or close-to-main stem reaches as artifical paths and not rivers.  Artifical paths are only supposed to be for throughflow lines in lakes/wetlands/reservoirs.
<br><br>
So, I identified reaches classed as 'ArtificalPath' that had no corresponding waterbodyID in the lakes dataset, assumed main stem reaches are perrenial, and reclassified those reaches as perrenial rivers.
<br><br> Canals and NHD's 'connectors' are listed as Artifical Channels.  'Connectors' are reaches the NHD had to add to make the network continuous in some places (i.e. through dams)
<br><br>
I am using the NHD's defintion of river intermittncy.  It's unclear how they designate perrenial/intermittent/ephemeral (no ephemeral reaches had measurements on them)
</p>

<h3><font color=darkcyan> Functions </font></h3>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.cluster import DBSCAN 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize 
from sklearn.decomposition import PCA

data = pd.read_csv('for_dbscan.csv', low_memory=False, encoding='latin-1')

<h4> <font color=darkolivegreen> Machine Learning: DBSCAN Clustering </font></h4>

In [23]:
sample = data #data.sample(n=100000)
var = sample[['chan_width','SLOPE', 'StreamOrde','DistDwnstrm', 'chan_depth', 'chan_velocity', 'unitPower', 'r', 'FCODEnorm', 'DASqKm', 'Fb', 'shearStress', 'minEntrain', 'TOTMA']]

# Scaling the data to bring all the attributes to a comparable level 
scaler = StandardScaler() 
X_scaled = scaler.fit_transform(var) 
  
# Normalizing the data so that the data 
# approximately follows a Gaussian distribution 
X_normalized = normalize(X_scaled) 
  
# Converting the numpy array into a pandas DataFrame 
X_normalized = pd.DataFrame(X_normalized) 
  
# Renaming the columns 
X_normalized.columns = var.columns 
X_normalized.head() 

#run algorithm
pca = PCA(n_components = 5) 
X_principal = pca.fit_transform(X_normalized) 
X_principal = pd.DataFrame(X_normalized) 

# Numpy array of all the cluster labels assigned to each data point 
db_default = DBSCAN(eps = 0.13, min_samples = 1000).fit(X_normalized) 
labels = db_default.labels_ 

In [24]:
from sklearn import metrics

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

X_principal['cluster'] = labels

# Plotting P1 on the X-Axis and P2 on the Y-Axis  
# according to the colour vector defined 
#fig, ax = plt.subplots()
#plt.scatter(X_principal[1], X_principal[2], c = X_principal['cluster']) 
#ax.legend()
  
#plt.show() 

Estimated number of clusters: 11
Estimated number of noise points: 163569


In [32]:
data['cluster'] = labels
data['logchan_width'] = np.log(data['SLOPE'])
data.groupby('cluster')['logchan_width'].describe()

#data.to_csv('clustered_ep_13.csv')

KeyError: 'Slope'